In [1]:
import pandas as pd
import numpy as np
from scipy.stats import binomtest, chi2, binom
import statsmodels.api as sm

# Settings
ALPHA = 0.05   # VaR quantile (e.g., 5% => expecting 5% exceptions)
LAGS = 10       # number of lags for the Dynamic Quantile test


def traffic_light_zone(n_exceptions, T, alpha):
    """Determine Basel traffic light zone based on number of exceptions."""
    cdf_value = binom.cdf(n_exceptions, T, alpha)
    
    if cdf_value >= 0.9999:
        return "Red"
    elif cdf_value >= 0.95:
        return "Yellow"
    else:
        return "Green"


def kupiec_pof_test(hits, T, n_exceptions, alpha):
    """Kupiec's Proportion of Failures (POF) Test."""
    if n_exceptions in [0, T]:
        return np.nan, np.nan
    
    L0 = (1 - alpha)**(T - n_exceptions) * (alpha)**(n_exceptions)
    p_hat = n_exceptions / T
    L1 = (1 - p_hat)**(T - n_exceptions) * (p_hat)**(n_exceptions)
    LR_uc = -2 * (np.log(L0) - np.log(L1))
    p_value = 1 - chi2.cdf(LR_uc, df=1)
    
    return LR_uc, p_value


def kupiec_tuff_test(first_failure_day, alpha):
    """Kupiec's Time Until First Failure (TUFF) Test."""
    if first_failure_day is None:
        return np.nan, np.nan
    
    n = first_failure_day
    num = alpha * (1 - alpha)**(n - 1)
    denom = (1.0 / n) * (1.0 - 1.0 / n)**(n - 1)
    
    if num <= 0 or denom <= 0:
        return np.nan, np.nan
    
    LR_TUFF = -2.0 * np.log(num / denom)
    p_value = 1.0 - chi2.cdf(LR_TUFF, df=1)
    
    return LR_TUFF, p_value


def christoffersen_cc_test(hits, T, LR_uc):
    """Christoffersen's Conditional Coverage Test."""
    if T < 2 or np.isnan(LR_uc):
        return np.nan
    
    I_prev = hits[:-1]
    I_curr = hits[1:]
    
    # Count transitions
    n00 = np.sum((I_prev == 0) & (I_curr == 0))
    n01 = np.sum((I_prev == 0) & (I_curr == 1))
    n10 = np.sum((I_prev == 1) & (I_curr == 0))
    n11 = np.sum((I_prev == 1) & (I_curr == 1))
    
    # Calculate transition probabilities
    pi0 = n01 / (n00 + n01) if (n00 + n01) > 0 else 0
    pi1 = n11 / (n10 + n11) if (n10 + n11) > 0 else 0
    
    total_ex = n01 + n11
    T1 = T - 1
    pi = total_ex / T1 if T1 > 0 else 0
    
    # Calculate likelihood ratios
    L_null = (1 - pi)**(n00 + n10) * pi**(n01 + n11)
    L_ind = ((1 - pi0)**n00 * pi0**n01) * ((1 - pi1)**n10 * pi1**n11)
    
    if L_ind == 0 or L_null == 0:
        return np.nan
    
    LR_ind = -2 * np.log(L_null / L_ind)
    LR_cc = LR_uc + LR_ind
    
    return 1 - chi2.cdf(LR_cc, df=2)


def haas_tbf_test(exception_days, alpha):
    """Haas's Time Between Failures (TBF) Test."""
    x = len(exception_days)
    if x == 0:
        return np.nan, np.nan
    
    # Compute intervals n_i
    intervals = [exception_days[0]]  # n_1 = t1
    for i in range(1, x):
        intervals.append(exception_days[i] - exception_days[i - 1])
    
    # Calculate LR_TBFI
    LR_TBFI = 0.0
    for n_i in intervals:
        num = alpha * (1 - alpha)**(n_i - 1)
        denom = (1.0 / n_i) * (1.0 - 1.0 / n_i)**(n_i - 1)
        
        if num <= 0 or denom <= 0:
            return np.nan, np.nan
        
        LR_TBFI += -2.0 * np.log(num / denom)
    
    # p-value for TBFI alone
    p_TBFI = 1.0 - chi2.cdf(LR_TBFI, df=x)
    
    return LR_TBFI, p_TBFI


def dynamic_quantile_test(hits, T, alpha, lags):
    """Dynamic Quantile (DQ) Test."""
    if T <= lags:
        return np.nan
    
    # Regress (hit indicator minus alpha) on a constant + lags of the hit indicator
    D = hits[lags:] - alpha
    Z = np.ones((len(D), lags + 1))  # constant + lags
    
    # Fill in lag values
    for t in range(lags, T):
        for lag in range(1, lags + 1):
            Z[t - lags, lag] = hits[t - lag]
    
    model_dq = sm.OLS(D, Z).fit()
    R2 = model_dq.rsquared
    T_dyn = len(D)
    DQ_stat = T_dyn * R2
    df_dq = Z.shape[1]  # number of regressors (including constant)
    
    return 1 - chi2.cdf(DQ_stat, df=df_dq)


def run_backtesting_tests(df, alpha=ALPHA, lags=LAGS):
    """Run all VaR backtesting tests on a dataframe."""
    df = df.sort_values(by="Date").reset_index(drop=True)
    T = len(df)
    
    # Define "hits": 1 if target < predicted VaR => exception
    hits = (df["target"] < df["prediction"]).astype(int).values
    n_exceptions = hits.sum()
    
    # 1) Binomial Test
    p_binom = binomtest(n_exceptions, T, alpha, alternative='two-sided').pvalue
    
    # 2) Traffic Light Test
    traffic_zone = traffic_light_zone(n_exceptions, T, alpha)
    
    # 3) Kupiec POF Test
    LR_uc, p_kupiec = kupiec_pof_test(hits, T, n_exceptions, alpha)
    
    # 4) Kupiec TUFF Test
    first_exception_idx = np.where(hits == 1)[0]
    if len(first_exception_idx) == 0:
        tuff_stat, p_tuff = np.nan, np.nan
    else:
        first_failure_day = first_exception_idx[0] + 1  # 1-based
        tuff_stat, p_tuff = kupiec_tuff_test(first_failure_day, alpha)
    
    # 5) Christoffersen CC Test
    p_christoffersen = christoffersen_cc_test(hits, T, LR_uc)
    
    # 6) Haas TBF Test
    exception_days = np.where(hits == 1)[0] + 1  # 1-based indexing
    LR_TBFI, p_TBFI = haas_tbf_test(exception_days, alpha)
    
    # Calculate combined Kupiec-Haas statistic
    if np.isnan(LR_uc) or np.isnan(LR_TBFI):
        LR_TBF, p_TBF = np.nan, np.nan
    else:
        LR_TBF = LR_uc + LR_TBFI
        p_TBF = 1.0 - chi2.cdf(LR_TBF, df=(n_exceptions + 1))
    
    # 7) Dynamic Quantile Test
    p_dynamic = dynamic_quantile_test(hits, T, alpha, lags)
    
    # Collect all results
    return {
        "n_obs": T,
        "n_exceptions": n_exceptions,
        "p_binom": p_binom,
        "traffic_light_zone": traffic_zone,
        "p_kupiec": p_kupiec,
        "tuff_stat": tuff_stat,
        "p_tuff": p_tuff,
        "p_christoffersen": p_christoffersen,
        "LR_TBFI": LR_TBFI,
        "p_TBFI": p_TBFI,
        "LR_TBF": LR_TBF,
        "p_TBF": p_TBF,
        "p_dynamic": p_dynamic
    }


# Main code execution directly in the script
# Load and filter data
results = pd.read_csv("results_lstm_lstm_dense.csv")
results['Date'] = pd.to_datetime(results['Date'], errors='coerce')
test_set = results[results["Set"] == "Test"].copy()

# Run tests for each ticker
tickers = test_set["Ticker"].unique()
results_list = []

for ticker in tickers:
    df_ticker = test_set[test_set["Ticker"] == ticker].copy()
    if df_ticker.empty:
        continue
    
    test_stats = run_backtesting_tests(df_ticker, alpha=ALPHA, lags=LAGS)
    test_stats["Ticker"] = ticker
    results_list.append(test_stats)

# Create and save results dataframe
test_results_df = pd.DataFrame(results_list)
print(test_results_df)

# Save results to CSV
test_results_df.to_csv("var_backtesting_results_test_set.csv", index=False)
print("Backtesting test results saved to var_backtesting_results_test_set.csv")

    n_obs  n_exceptions   p_binom traffic_light_zone  p_kupiec  tuff_stat  \
0    1023            42  0.221910              Green  0.176118   0.120168   
1    1023            55  0.565788              Green  0.585109   0.413084   
2    1023            48  0.719724              Green  0.648106   5.991465   
3    1023            53  0.773989              Green  0.791858   1.800543   
4    1023            67  0.026054             Yellow  0.029631   0.011307   
5     845            49  0.269613              Green  0.298233   3.321462   
6    1023            55  0.565788              Green  0.585109  -0.000000   
7    1023            56  0.472901              Green  0.492879   0.413084   
8    1023            55  0.565788              Green  0.585109   2.377553   
9    1023            53  0.773989              Green  0.791858   0.120168   
10   1023            43  0.281240              Green  0.229850   1.097663   
11   1023            42  0.221910              Green  0.176118   2.961289   

In [2]:
test_results_df.round(3).sort_values(by='p_christoffersen')

,n_obs,n_exceptions,p_binom,traffic_light_zone,p_kupiec,tuff_stat,p_tuff,p_christoffersen,LR_TBFI,p_TBFI,LR_TBF,p_TBF,p_dynamic,Ticker
30,1023,59,0.251,Green,0.271,2.961,0.085,0.015,54.204,0.653,55.415,0.644,0.063,SCYR.MC
4,1023,67,0.026,Yellow,0.030,0.011,0.915,0.038,64.977,0.547,69.708,0.420,0.690,ANA.MC
32,1023,35,0.018,Green,0.014,0.640,0.424,0.039,38.238,0.325,44.246,0.163,0.839,TEF.MC
23,1023,36,0.031,Green,0.022,0.413,0.520,0.060,46.210,0.119,51.455,0.057,0.633,NTGY.MC
16,1023,44,0.350,Green,0.294,3.321,0.068,0.080,44.912,0.433,46.014,0.430,0.885,IBE.MC
25,1023,48,0.720,Green,0.648,3.321,0.068,0.085,36.529,0.887,36.737,0.902,0.508,RED.MC
18,1023,48,0.720,Green,0.648,1.098,0.295,0.085,31.331,0.970,31.539,0.975,0.540,ITX.MC
13,1023,47,0.615,Green,0.546,1.098,0.295,0.086,34.795,0.906,35.159,0.916,0.654,FER.MC
19,1023,39,0.085,Green,0.069,0.049,0.825,0.100,63.613,0.008,66.911,0.005,0.195,LOG.MC
28,1023,63,0.098,Yellow,0.100,1.535,0.215,0.146,81.843,0.056,84.543,0.044,0.841,SAB.MC
